In [3]:

import pandas as pd
import numpy as np
import time
import seaborn as sb
import datetime as dt
import matplotlib.pyplot as plt
from collections import namedtuple

In [4]:

trade = namedtuple('trade', 'Side Entry Exit Profit')
def format_date(t):
    if len(t)<11:
        t = t+' 12:00' #somehow the data is missing the 12pm timem stamp
    date,time = t.split(' ')
    month,day,year = date.split('/')
    hr,sec = time.split(':')

    return dt.datetime(int(year),int(month),int(day),int(hr),int(sec))


In [101]:
def gap_tester(data, TP, SL):
    current_day = None
    volume = {}
    entry_price = 0
    close_price = 0
    side = -1#1 indicates buy and 0 indicates sell
    win_list = [] 
    lose_list = []
    gaps=[]
    last_price = 0
    prev_last = 0
    tp_price  = 0 
    for index, row in data.iterrows():
        
        date = format_date(row['Date'])
        if date.hour>15:
            continue

        if date.hour<9: 
            continue

        if date.hour == 9 and date.minute<30:
            continue
            

        
        if current_day == None or date.day != current_day: # new data reset everyhing
            if last_price:
                
                prev_last = last_price
                if row['Low'] > prev_last:
                    gaps.append([prev_last,row['Low'],'up']) # up
                elif row['High']<prev_last:
                    gaps.append([row['High'],prev_last,'down']) #down
            current_day = date.day

        last_price = row['Close']


        if prev_last == 0:
            continue
       #only if no trade to trade
                
        to_remove = []
        
        for gap in gaps:
            if row['Open']<=gap[0]: # below the gap now
                if row['High']>gap[0]: # cross the gap buy
                    if side  == -1: 

                        side = 1
                        entry_price = gap[0]
                        tp_price = gap[1]
                        sl_price = entry_price*(1-SL)

                    if row['High']<gap[1]: # adjust the bootom of the gap
                        gap[0] = row['High']
                    else:
                        to_remove.append(gap)
            elif row['Open']>=gap[1]:
                if row['Low']<gap[1]: # sell
                    if side  == -1:

                        side = 0
                        entry_price = gap[1]
                        tp_price = gap[0]
                        sl_price = entry_price*(1+SL)

                    if row['Low']>gap[0]: # adjust the top of the gap
                        gap[1] = row['Low']
                    else:
                        to_remove.append(gap)

        if to_remove:

            for remove in to_remove:
                gaps.remove(remove)

                    
                    
        if entry_price != 0:
            if side == 1:# buy
                if row['High']>tp_price:
                    #add to wining trade
                    win_list.append(trade(side,entry_price,tp_price,(tp_price-entry_price)/entry_price))
                    entry_price = 0
                    side =-1
                    

                elif row['Low']<sl_price: #price less than SL
                    
                    lose_list.append(trade(side,entry_price,sl_price,(entry_price- sl_price)/entry_price))

                    entry_price = 0
                    side =-1
                else:
                    continue
            elif side ==0: # sell
                if row['Low']<tp_price: #price lower than TP:
  
                    win_list.append(trade(side,entry_price,tp_price,(entry_price-tp_price)/entry_price))

                    entry_price = 0
                    side =-1
              
                elif row['High']>sl_price: #price less than SL
                    
                    #add to losing trade
                    lose_list.append(trade(side,entry_price,sl_price,(sl_price-entry_price)/entry_price))
                    entry_price = 0
                    side =-1
  

                else:
                    continue
    return win_list,lose_list

In [105]:

#test case should have one trade sell. SL activated
print("test_case 1")
day1 = '4/24/2020'
day2 ='4/27/2020'
data = pd.read_csv('SPY.csv')
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data = pd.concat([data_day1,data_day2], axis=0)

print(gap_tester(data,0.002,0.002))

#test case should 1 trade sell @ profit
print("test_case2")
day0 ='4/24/2020'
day1 = '4/27/2020'
day2 ='4/28/2020'
data = pd.read_csv('SPY.csv')
test_data0 = data.Date.str.contains(day0)
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data_day0=data[test_data0]
data = pd.concat([data_day0, data_day1,data_day2], axis=0)


print(gap_tester(data,0.005,0.005))
# no trades
print("test_case3")
day1 = '4/1/2020'
day2 ='4/2/2020'
data = pd.read_csv('SPY.csv')

test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data = pd.concat([data_day1,data_day2], axis=0)
print(gap_tester(data,0.005,0.005))

#test case 4 consecutive day
print("test_case4")
day0 ='4/20/2020'
day1 = '4/21/2020'
day2 ='4/22/2020'
data = pd.read_csv('SPY.csv')
test_data0 = data.Date.str.contains(day0)
test_data = data.Date.str.contains(day1)
test_data2= data.Date.str.contains(day2)
data_day1=data[test_data]
data_day2=data[test_data2]
data_day0=data[test_data0]
data = pd.concat([data_day0, data_day1,data_day2], axis=0)

print(gap_tester(data,0.005,0.005))


test_case 1
([], [trade(Side=0, Entry=284.81, Exit=285.37962, Profit=0.001999999999999952)])
test_case2
([trade(Side=0, Entry=290.98, Exit=287.1, Profit=0.013334249776616933)], [trade(Side=0, Entry=284.81, Exit=286.23404999999997, Profit=0.0049999999999998795)])
test_case3
([], [])
test_case4
([], [trade(Side=1, Entry=277.14, Exit=275.7543, Profit=0.004999999999999949), trade(Side=1, Entry=277.3, Exit=275.9135, Profit=0.005000000000000044), trade(Side=0, Entry=278.14, Exit=279.53069999999997, Profit=0.0049999999999999324)])


In [106]:
files = ['TLT.csv','SPY.csv','APPL.CSV', 'IWM.csv' ,'AMZN.csv']
for name in files:
    data = pd.read_csv(name)
    win_list,lose_list  = gap_tester(data,0.002,0.004)
    total_win = 0
    total_lose =0 
    for i in win_list:
        assert(i.Profit>=0)
        total_win +=i.Profit
    for i in lose_list:
        assert(i.Profit>=0)
        total_lose +=i.Profit

    print("total of {} wining trades at a {} gain".format(str(len(win_list)),str(total_win)))
    print("total of {} losing trades at a {} loss".format(str(len(lose_list)),str(total_lose)))

total of 89 wining trades at a 0.4355601914264174 gain
total of 120 losing trades at a 0.4800000000000014 loss
total of 89 wining trades at a 0.3366121793809623 gain
total of 130 losing trades at a 0.5199999999999994 loss
total of 93 wining trades at a 0.4839353011879471 gain
total of 169 losing trades at a 0.6760000000000003 loss
total of 106 wining trades at a 0.5481759964942332 gain
total of 152 losing trades at a 0.6079999999999997 loss
total of 109 wining trades at a 0.5433125271238087 gain
total of 118 losing trades at a 0.4720000000000005 loss


In [42]:
a.remove(b)